<a href="https://colab.research.google.com/github/Camouflage10/MEME-GAN/blob/main/GAN/gan_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from keras import Sequential
from keras.layers import (
    Dense,
    Conv2D,
    Conv2DTranspose,
    MaxPooling2D,
    Flatten,
    Dropout,
    LeakyReLU,
    InputLayer,
    Reshape,
)

In [2]:
def discriminator():
    depth = 64
    model = Sequential(
        [
            InputLayer((100, 100, 3)),
            # Conv 1
            Conv2D(depth * 1, (3, 3), strides=(2, 2), padding="same"),
            LeakyReLU(0.2),
            # Conv 2
            Conv2D(depth * 2, (3, 3), strides=(2, 2), padding="same"),
            LeakyReLU(0.2),
            # Conv 3
            Conv2D(depth * 4, (3, 3), strides=(2, 2), padding="same"),
            LeakyReLU(0.2),
            # Connected
            Flatten(),
            Dropout(0.25),
            Dense(1, activation="sigmoid"),  # output - T or F
        ]
    )
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model


discriminator().summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 50, 50, 64)        1792      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 50, 50, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 25, 128)       73856     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 25, 25, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 256)       295168    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 13, 13, 256)       0         
                                                                 
 flatten (Flatten)           (None, 43264)             0

In [3]:
def generator(latent_dim):
  n_nodes = 25 * 25 * 128
  model = Sequential([
    # 25 x 25 base image
    Dense(n_nodes, input_dim=latent_dim),
    LeakyReLU(0.2),
    Reshape((25, 25, 128)),
    # upsample to 50 x 50
    Conv2DTranspose(128, (4, 4), strides=(2,2), padding='same'),
    LeakyReLU(0.2),
    # up sample to 100 x 100
    Conv2DTranspose(128, (4, 4), strides=(2,2), padding='same'),
    LeakyReLU(0.2),
    Conv2D(3, (100, 100), activation='tanh', padding='same'),
  ])
  return model

generator(100).summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 80000)             8080000   
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 80000)             0         
                                                                 
 reshape (Reshape)           (None, 25, 25, 128)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 50, 50, 128)      262272    
 nspose)                                                         
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 50, 50, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 100, 100, 128)    262272    
 ranspose)                                            

In [4]:
def gan(g_model, d_model):
  d_model.trainable = False
  model = Sequential([
    g_model,
    d_model,
  ])
  model.compile(loss='binary_crossentropy', optimizer='adam')
  return model

gan(generator(100), discriminator()).summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 100, 100, 3)       12444547  
                                                                 
 sequential_3 (Sequential)   (None, 1)                 414081    
                                                                 
Total params: 12,858,628
Trainable params: 12,444,547
Non-trainable params: 414,081
_________________________________________________________________


In [5]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = np.random.randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = np.ones((n_samples, 1))
	return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = np.random.randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = np.zeros((n_samples, 1))
	return X, y

In [6]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=10):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(n * n):
		# define subplot
		plt.subplot(n, n, 1 + i)
		# turn off axis
		plt.axis('off')
		# plot raw pixel data
		plt.imshow(examples[i])
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	plt.savefig(filename)
	plt.close()

In [7]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

In [8]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=128):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = np.ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		# if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# load dataset
# path = "../Data/Processed_Data/Drake_Hotline_Bling/imgs/"
path = '/content/drive/MyDrive/GAN_Data/Drake_Hotline_Bling/imgs/'
raw_images = []
for filename in os.listdir(path):
    img = cv2.imread(path + filename)
    raw_images.append(img)

dataset = np.array(raw_images)
print(dataset.min(), dataset.max())

scale = lambda x: (x - 127.5) / 127.5
dataset = scale(dataset)

print(dataset.min(), dataset.max())
print(dataset.shape)


0 255
-1.0 1.0
(536, 100, 100, 3)


In [11]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


In [ ]:
latent_dim = 100
d_model = discriminator()
g_model = generator(latent_dim)
gan_model = gan(g_model, d_model)
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=5, n_batch=25)